# Dataset preparation

### Fernando Fincatti | Laís Nascimento

---
<font color=blue, size="4">  __A quick overview of the dataset__
    
--- 

In [1]:
import pandas as pd

In [2]:
#LENDO O DATASET QUE SERÁ UTILIZADO
data = pd.read_excel("data.xlsx", sheet_name="fernando")

In [3]:
#ESTE DATASET CONTÊM APENAS AS MANCHETES RELAVANTES PROVENIENTES DA BASE OBC
relevantes = pd.read_excel("ManchetesRelevantes.xlsx")

In [4]:
data.head()

,Data,De,HTML,Resumo
0,2020-04-03 23:46:37,googlealerts-noreply@google.com,"<html lang=""pt-BR""> <head> </head> <body> <div...",Google Bonos del Impacto Social Atualização as...
1,2020-04-04 01:10:54,googlealerts-noreply@google.com,"<html lang=""pt-BR""> <head> </head> <body> <div...",Google contratos de impacto social Atualização...
2,2020-04-04 01:09:50,googlealerts-noreply@google.com,"<html lang=""pt-BR""> <head> </head> <body> <div...",Google outcome-based contract Atualização diár...
3,2020-04-04 01:16:32,googlealerts-noreply@google.com,"<html lang=""pt-BR""> <head> </head> <body> <div...",Google development impact bond Atualização ass...
4,2020-04-04 01:18:02,googlealerts-noreply@google.com,"<html lang=""pt-BR""> <head> </head> <body> <div...",Google outcomes-based contracts Atualização as...


---
<font color=blue, size="4">  __We need to treat the data__
    
--- 

For best result, is necessary treat the data. The processes used were:
    
    1 - Every letter was transform to lowercase letters
    
    2 - The punctuation was disregarded ([\ / ! - . : ? ; ])
    
    3 - We used a text translator to the news in another languege, this way every single news is in English
    
These procedures were used to make the news belong to the same universe, in other words, the model have more information to work with.

---

In [5]:
import re

In [6]:
def cleanup(text):
    import string
    punctuation = '[\/!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

In [7]:
from textblob import TextBlob

---
For the translator was used the textblob library, to install open Anaconda Prompt and use the command below:

$ pip install -U textblob

Link: https://kanoki.org/2019/11/06/python-detect-and-translate-language/

--- 

---
<font color=blue, size="4">  __Extracting the news from the email's HTML__
    
--- 

In [8]:
from tqdm.auto import tqdm

manchetes = {}
contador = 0
email=[]
lista_manchetes=[]
numero_atual = 0
loop = tqdm(total = data.shape[0], position = 0, leave = False)
for i in data.HTML.index:
    loop.set_description("Extraindo manchetes...".format(numero_atual))
    loop.update(1)
    numero_atual += 1
    texto = str(data.HTML[i])
    
    for indice in range(len(texto)):
        
        if texto[indice:indice+7] == "message":
            
            manchete_incial = texto[indice+11:indice+150]
            achou = False
            manchete = " "
            indice_letra = 0
            email.append(i+2)
            
            while indice_letra < len(manchete_incial):
                
                if manchete_incial[indice_letra] == "}":
                    
                    manchete = manchete_incial[0:indice_letra-6]
                    achou = True
                    manchetes[contador] = manchete

                    manchete_limpa = cleanup(manchete)
                    manchete_min = manchete_limpa.lower()

                    try:
                        hi_blob = TextBlob(manchete_min)
                        manchete_pronta = hi_blob.translate(to='en')
                    except:
                        manchete_pronta = manchete_min

                    lista_manchetes.append(str(manchete_pronta))
                    contador += 1
                    break
                        
                else:
                    
                    indice_letra += 1
loop.close()

In [9]:
lista_links = []
for linha in data.HTML.index:   #Percorre cada linha do dataset
    texto = data.HTML[linha]
    
    contador_de_titulos = 0
    for indice in range(len(texto)-7):
        if texto[indice:indice+7] == "message":
            contador_de_titulos += 1
    
    
    
    indice = 0
    texto_com_url = ""
    
    while indice < len(texto):  #Percorre o texto dentro da linha
        if texto[indice:indice + 7] == "widgets":
            while True:
                if texto[indice+5:indice+8] == "} ]":
                    break
                else:
                    texto_com_url += texto[indice+8]
                    indice += 1
            break 
        else:
            indice += 1
            
    
    indice = 0
    numero_atual_de_titulos = 0
    
    while indice < len(texto_com_url) and numero_atual_de_titulos < contador_de_titulos:
        if texto_com_url[indice : indice+3] == "url":
            link = ""
            while True:
                if texto_com_url[indice+7] == '"':
                    numero_atual_de_titulos += 1
                    break
                else:
                    link += texto_com_url[indice+7]
                    indice += 1
            lista_links.append(link)
        else:
            indice += 1
        
    
    

In [10]:
indice_lista = 0
while indice_lista < len(lista_links):
    contador_de_https = 0
    contador = 0
    link = lista_links[indice_lista]
    while contador < len(link):
        if link[contador:contador+5] == "https" or link[contador:contador+4] == "http":
            contador_de_https += 1
        if contador_de_https == 2:
            link_completo = link[contador:len(link)]
            link_final = ""
            indice_google = 0
            while indice_google < len(link_completo):
                
                if link_completo[indice_google : indice_google+7] == "u0026ct":
                    break
                else:
                    link_final += link_completo[indice_google]
                    indice_google += 1
                
            lista_links[indice_lista] = link_final
            break
        contador += 1
    
    indice_lista += 1
    
    
print(len(lista_links)) 

366


In [11]:
len(lista_manchetes)

366

---
<font color=blue, size="4">  __Adding the relevant news__
    
--- 

In [12]:
lista_backup = []

In [13]:
loop = tqdm(total = relevantes.shape[0], position = 0, leave = False)
numero_atual = 0
for i in relevantes.index:
    loop.set_description("Manchetes base OBC...".format(numero_atual))
    loop.update(1)
    numero_atual += 1
    
    manchete_relevante = str(relevantes["Manchetes da Base de Dados"][i])
    manchete_relevante_limpa = cleanup(manchete_relevante)
    manchete_relevante_min = manchete_relevante_limpa.lower()
    
    try:
        manchete_relevante_blob = TextBlob(manchete_relevante_min)
        manchete_relevante_traduzida = manchete_relevante_blob.translate(to='en')
    except:
        manchete_relevante_traduzida = manchete_relevante_min
        
    lista_manchetes.append(str(manchete_relevante_traduzida))
    lista_backup.append(str(manchete_relevante_traduzida))
    lista_links.append("Não se aplica")

loop.close()

In [14]:
dicionario = {}
    
dicionario['Manchetes'] = lista_manchetes
#dicionario['Indice'] = email
dicionario['Link'] = lista_links

In [15]:
data_manchetes = pd.DataFrame(data=dicionario)

In [16]:
data_manchetes.head()

,Manchetes,Link
0,bono de anses cómo evitar las aglomeraciones ...,https://www.infobae.com/economia/2020/04/03/bo...
1,la ampliación del bono social costará 23 millo...,https://m.europapress.es/economia/energia-0034...
2,bono de 380 soles beneficiarios podrán retira...,https://larepublica.pe/economia/2020/03/31/bon...
3,está em vigor mp que permite suspensão de cont...,https://www.conjur.com.br/2020-abr-02/medida-p...
4,covid 19 a força maior e seus potenciais impa...,https://www.portosenavios.com.br/artigos/artig...


In [17]:
result = pd.concat([data, data_manchetes], ignore_index=True, axis=1)

In [18]:
result.columns = ['Data','De','HTML','Resumo','Manchete','Link']

In [19]:
result["Relevância"] = int(0)

In [20]:
result.head()

,Data,De,HTML,Resumo,Manchete,Link,Relevância
0,2020-04-03 23:46:37,googlealerts-noreply@google.com,"<html lang=""pt-BR""> <head> </head> <body> <div...",Google Bonos del Impacto Social Atualização as...,bono de anses cómo evitar las aglomeraciones ...,https://www.infobae.com/economia/2020/04/03/bo...,0
1,2020-04-04 01:10:54,googlealerts-noreply@google.com,"<html lang=""pt-BR""> <head> </head> <body> <div...",Google contratos de impacto social Atualização...,la ampliación del bono social costará 23 millo...,https://m.europapress.es/economia/energia-0034...,0
2,2020-04-04 01:09:50,googlealerts-noreply@google.com,"<html lang=""pt-BR""> <head> </head> <body> <div...",Google outcome-based contract Atualização diár...,bono de 380 soles beneficiarios podrán retira...,https://larepublica.pe/economia/2020/03/31/bon...,0
3,2020-04-04 01:16:32,googlealerts-noreply@google.com,"<html lang=""pt-BR""> <head> </head> <body> <div...",Google development impact bond Atualização ass...,está em vigor mp que permite suspensão de cont...,https://www.conjur.com.br/2020-abr-02/medida-p...,0
4,2020-04-04 01:18:02,googlealerts-noreply@google.com,"<html lang=""pt-BR""> <head> </head> <body> <div...",Google outcomes-based contracts Atualização as...,covid 19 a força maior e seus potenciais impa...,https://www.portosenavios.com.br/artigos/artig...,0


---
<font color=blue, size="4">  __Adding the relevance column__
    
--- 

In [21]:
data_backup = pd.read_excel("data.xlsx", sheet_name="backup")

In [22]:
loop = tqdm(total = data_backup.shape[0], position = 0, leave=False)
numero_atual = 0 
for i in data_backup.index:
    loop.set_description("Adicionando manchetes relevantes...".format(numero_atual))
    loop.update(1)
    
    numero_atual += 1 
    
    
    manchete_d = str(data_backup.Manchetes[i])
    manchete_d_limpa = cleanup(manchete_d)
    manchete_d_min = manchete_d_limpa.lower()
    
    try:
        manchete_d_min_blob = TextBlob(manchete_d_min)
        manchete_d_traduzida = manchete_d_min_blob.translate(to='en')
    except:
        manchete_d_traduzida = manchete_d_min
    
    lista_backup.append(str(manchete_d_traduzida))
    
loop.close()

In [23]:
rel = 0
for i in result.index:
    if result["Manchete"][i] in lista_backup:
        result.Relevância[i] = int(1)
        rel += 1

C:\Users\Fernando\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [24]:
result.Relevância.value_counts(True)

1    0.50229
0    0.49771
Name: Relevância, dtype: float64

In [25]:
result.head()

,Data,De,HTML,Resumo,Manchete,Link,Relevância
0,2020-04-03 23:46:37,googlealerts-noreply@google.com,"<html lang=""pt-BR""> <head> </head> <body> <div...",Google Bonos del Impacto Social Atualização as...,bono de anses cómo evitar las aglomeraciones ...,https://www.infobae.com/economia/2020/04/03/bo...,0
1,2020-04-04 01:10:54,googlealerts-noreply@google.com,"<html lang=""pt-BR""> <head> </head> <body> <div...",Google contratos de impacto social Atualização...,la ampliación del bono social costará 23 millo...,https://m.europapress.es/economia/energia-0034...,0
2,2020-04-04 01:09:50,googlealerts-noreply@google.com,"<html lang=""pt-BR""> <head> </head> <body> <div...",Google outcome-based contract Atualização diár...,bono de 380 soles beneficiarios podrán retira...,https://larepublica.pe/economia/2020/03/31/bon...,0
3,2020-04-04 01:16:32,googlealerts-noreply@google.com,"<html lang=""pt-BR""> <head> </head> <body> <div...",Google development impact bond Atualização ass...,está em vigor mp que permite suspensão de cont...,https://www.conjur.com.br/2020-abr-02/medida-p...,0
4,2020-04-04 01:18:02,googlealerts-noreply@google.com,"<html lang=""pt-BR""> <head> </head> <body> <div...",Google outcomes-based contracts Atualização as...,covid 19 a força maior e seus potenciais impa...,https://www.portosenavios.com.br/artigos/artig...,0


In [26]:
result.shape

(655, 7)

In [ ]:
print("DATASET PRONTO PARA USO!")